# 5. Macro and Sentiment Features
Create features from FRED macro data and aggregate news sentiment

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
from utils.feature_functions import forward_fill_macro, aggregate_sentiment
from utils.hopsworks_helpers import get_feature_store, create_feature_group
import yaml

# Load config
with open('../config/config.yaml', 'r') as f:
    config = yaml.safe_load(f)

## Load Raw Data from Hopsworks

In [ ]:
fs = get_feature_store()

# Load FRED data
treasury_fg = fs.get_feature_group('treasury_yield_raw', version=1)
cpi_fg = fs.get_feature_group('cpi_raw', version=1)

treasury_df = treasury_fg.read()
cpi_df = cpi_fg.read()

# Load news sentiment
news_fg = fs.get_feature_group('news_sentiment_raw', version=1)
news_df = news_fg.read()

print(f"Treasury data: {treasury_df.shape}")
print(f"CPI data: {cpi_df.shape}")
print(f"News data: {news_df.shape}")

## Macro Features from FRED

In [ ]:
# Create complete date range (trading days)
# We'll use QQQ dates as the reference
qqq_fg = fs.get_feature_group('qqq_raw', version=1)
qqq_dates = qqq_fg.read()['date'].unique()
date_range = pd.DatetimeIndex(qqq_dates)

# Forward fill CPI to daily values
cpi_daily = forward_fill_macro(cpi_df, 'cpiaucsl', date_range)

# Calculate CPI year-over-year change
cpi_daily['cpi_yoy'] = cpi_daily['cpiaucsl'].pct_change(252)  # ~252 trading days in a year

# Treasury yield features
treasury_df['yield_change'] = treasury_df['dgs10'].diff()
treasury_df['yield_ma_20'] = treasury_df['dgs10'].rolling(20).mean()

# Merge macro features
macro_features = treasury_df.merge(cpi_daily, on='date', how='outer')
macro_features = macro_features.sort_values('date')

print(f"Macro features shape: {macro_features.shape}")
macro_features.head()

## Sentiment Features from News

In [ ]:
# Aggregate article-level sentiment to daily
sentiment_features = aggregate_sentiment(news_df, date_col='date')

print(f"Daily sentiment features shape: {sentiment_features.shape}")
sentiment_features.head()

## Upload to Hopsworks

In [ ]:
# Select final columns
macro_cols = ['date', 'dgs10', 'yield_change', 'yield_ma_20', 'cpiaucsl', 'cpi_yoy']
macro_features_final = macro_features[macro_cols]

# Create feature groups
macro_fg = create_feature_group(
    fs,
    name='macro_features',
    df=macro_features_final,
    primary_key=['date'],
    description='Macroeconomic features: Treasury yields, CPI'
)

sentiment_fg = create_feature_group(
    fs,
    name='sentiment_features',
    df=sentiment_features,
    primary_key=['date'],
    description='Daily aggregated news sentiment from FinBERT'
)

print("Macro and sentiment features uploaded to Hopsworks!")